# 외감법인(액티브+휴폐업) 사업자등록번호로 사업장 주소 크롤링
20221011
- kiscode=059782 나이스 db만의 키값인듯?

https://www.nicebizinfo.com/ep/EP0100M001GE.nice

https://www.nicebizinfo.com/ep/EP0100M002GE.nice?kiscode=059782

https://www.nicebizinfo.com/ep/EP0100M001GE.nice

https://www.nicebizinfo.com/ep/EP0100M002GE.nice?kiscode=373397
- https://www.nicebizinfo.com/ep/EP0100M001GE.nice?itgSrch=2208113826

In [33]:

import requests
from bs4 import BeautifulSoup

        
url='https://www.nicebizinfo.com/ep/EP0100M001GE.nice?itgSrch=2208113826'

headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}
res=requests.get(url,headers=headers)
soup=BeautifulSoup(res.text, 'html.parser')
print(res.status_code) #속성접근 
print(res.text)

200
<!-- COMMON TOP-UP -->
<!DOCTYPE html>
<html lang="ko">
<head>

<meta charset="utf-8">
<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">

<title>기업정보의 모든 것, 나이스비즈인포 - 기업신용분석보고서 사이트</title>

<link rel="canonical" href="https://www.nicebizinfo.com/" />
<link rel="alternate" href="https://m.nicebizinfo.com/" media="only screen and (max-width: 640px)" />

<meta name="title" content="기업정보의 모든 것, 나이스비즈인포 - 기업신용분석보고서 사이트" />
<meta name="writer" content="나이스평가정보" />
<meta name="description" content="기업정보의 모든 것, 나이스비즈인포에서 기업정보, 재무정보, 신용정보가 담긴 기업정보 분석보고서를 만나보세요. 신용분석, 신용등급, 기업신용, 보고서 사이트, 주요재무, 경쟁사 비교" />
<meta name="keywords" content="기업정보, 기업현황, 재무정보, 신용정보, 기업신용, 신용등급, 조기경보, 경쟁사비교분석, 인물정보, 부고, 요약기업정보, 
                                                상세기업정보, 특허정보,  신용보고서, 기업보고서, 기업리포트, 연봉현황, 종업원수, 산업분석, 주주정보, 관계사정보, 재무비율, 
                                                수익성, 안정성, 성장성, 활동성, 신용도, 기업평가, 매출액, 영업이익, 그룹정보, 사업자번호, 사업자등록번호, 법인번호, 대표자, 
                                 

In [30]:
soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN">
 <html> <head> <title>NICE Information Service</title> <meta content="no" http-equiv="imagetoolbar"/> </head> <body leftmargin="0" style="background:url('img/bg_01.gif') repeat-x;" topmargin="0"> <table border="0" cellpadding="0" cellspacing="0" height="100%" width="100%"> <tr> <td style="text-align:center; vertical-align:middle;"> <table border="0" cellpadding="0" cellspacing="0" width="700"> <tr> <td align="right" width="50%"><img src="img/img_ovj.gif"/></td> <td style="padding-bottom:33px;" width="50%"><img hspace="10" src="img/txt_403.gif"/></td> </tr> <tr> <td align="right" colspan="2"><img src="img/img_ci.gif"/></td> </tr> </table> </td> </tr> </table> </body> </html>

## 전체코드

In [50]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

- BIZ_NO 리스트로 가져오기

In [70]:
#휴폐업중소법인기업개요_산업분류.csv불러오기
import pandas as pd

close_dt = pd.read_csv("./data/1차정제/1_FIN/휴폐업_중소법인_기업개요_산업분류.csv",encoding='cp949')
close_dt['BIZ_NO'] =close_dt['BIZ_NO'].astype(str) # 문자열로 변환
#close_dt['close']='Y'
close_bizno=close_dt['BIZ_NO'].values.tolist()
close_bizno
print(len(close_dt['BIZ_NO']))#전체 휴폐업 기업수
print(close_bizno[1])

6739
1388101749


In [69]:
company_num=close_bizno#6739번 돌아감.
result=[]
for i in company_num:
    total_url='https://www.nicebizinfo.com/ep/EP0100M001GE.nice?itgSrch='+i
    headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}
    res=requests.get(total_url,headers=headers)
    soup=BeautifulSoup(res.text, 'html.parser')
    #예외처리
    try :      
        address=soup.select_one('.addr').text.strip()#주소
    except:
        address='NAN'#주소
    #print(address)
    try:
        company_nm=soup.select_one('span > a').text.strip()#기업명
    except:
        company_nm='NAN'
    #print(company_nm)
    #owner=soup.select_one('.ml25').text.strip('ㅣ').strip()#대표자
    #print(owner)
    try:
        industry=soup.select_one('.fll.textLeftBg.gear').text.strip()#산업
    except:
        industry='NAN'
    #print(industry)
    temp=[i,company_nm,address,industry]
    result.append(temp)
    #temp={'주소':address,'기업명':company_nm,'대표자':owner,'산업':industry}
    df = pd.DataFrame(result, columns = ['BIZ_NO','기업명','주소','산업'])
    #print(df)
df.to_excel('휴폐업_중소법인_주소.xlsx')


# 셀레니움으로 상세정보 가져오기
- 클릭하면 데이터가 생성되는 동적 html이므로 셀레니움 활용

### 데이터 확인

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
chrome_path='C:/Users/Choi/workspace/sesac/chromedriver_win32/chromedriver.exe'
driver = webdriver.Chrome(chrome_path) #data크롬창 뜸
#1268145633->상세기업정보 없음
#1388101749-> 상세기업정보 있음
#6218111642->외감법인액티브
driver.get('https://www.nicebizinfo.com/ep/EP0100M001GE.nice?itgSrch=6218111642') #비어있던 data창에 정글사이트 켜짐
driver.implicitly_wait(10) #로딩이 끝날동안 10초 대기


- '기업정보 상세보기' 버튼 클릭
- time.sleep :멈추고자하는 시간(초)만큼 프로그램이 일시정지입력한 시간이 지난 다음에 바로 그 다음줄 부터 프로그램이 다시 시작 됩니다. 출처: https://blockdmask.tistory.com/571 [개발자 지망생:티스토리]

In [12]:
#기업정보 상세보기 페이지가 없는 경우도 있음->예외처리
'''
from selenium.common.exceptions import TimeoutException, NoSuchElementException
try:
except NoSuchElementException:
    pass
'''

driver.find_element(By.CSS_SELECTOR, 'td.tar.bdr1').click() 
time.sleep(2)


- 웹드라이버로 연 페이지 소스 가져오기https://velog.io/@anjaekk/Selenium%EA%B3%BC-BeautifulSoup%EC%9D%84-%EC%9D%B4%EC%9A%A9%ED%95%9CWeb-ScrapingCrawling
- page_source = driver.page_source 

In [61]:
result=[]

from bs4 import BeautifulSoup
page_source = driver.page_source
soup = BeautifulSoup(page_source, "html.parser")

#사업자등록번호 i
#대표자 div.iconBox.bg1[1]
try :      
    owner=soup.select_one('div>strong').text.strip()
except:
    owner='NAN'#주소
#기업형태 
try :      
    company_type=soup.select_one('.iconBox.bg4>strong').text.replace('\n',"").replace(' ','')
except:
    company_type='NAN'
#본사주소
try :      
    address=soup.select_one('.iconBox.bg2>strong').text
except:
    address='NAN'
#산업
try :      
    industry=soup.select_one('.iconBox.bg5>strong').text
except:
    industry='NAN'
#설립일자
'''출력안됨...;;
try :      
    start_date=soup.select_one('.iconBox.bg3>strong')
except:
    start_date='NAN'
'''
    
#예상평균연봉
try :      
    expected_salary=soup.select('.fz16')[0].text.strip()
except:
    expected_salary='NAN'
#올해 입사자 평균연봉
try :      
    salary=soup.select('.fz16')[1].text.strip()
except:
    salary='NAN'
#종업원수
try :      
    employer=soup.select('.fz16')[2].text.strip()
except:
    employer='NAN'
#입사율
try :      
    in_rate=soup.select('.fz16')[3].text.replace('\n',"").replace(' ','')
except:
    in_rate='NAN'
#퇴사율
try :      
    out_rate=soup.select('.fz16')[4].text.replace('\n',"").replace(' ','')
except:
    out_rate='NAN'
#업력
try :      
    period=soup.select('.fz16')[5].text.replace('\n',"").replace(' ','')
except:
    period='NAN'
    
    
temp=[owner,company_type,address,industry,expected_salary,salary,employer,in_rate
     ,out_rate,period]
result.append(temp)
print(result)

[['정득부', '중소기업,외감', '경남 양산시 북정서길 3, 402호', '(C13225)직물포대 제조업', '4,000만원 이상 ~ 6,000만원 미만', '~ 4,000만원 미만', '1명 이상 ~ 20명 미만', '0.00%(0명)', '0.00%(0명)', '29년']]
